# Investigating Inhibitory and Excitatory pairs in the Hemibrain/Full brain data


In [2]:
from neuprint import Client, fetch_neurons

from neuprint import fetch_simple_connections, fetch_synapse_connections, NeuronCriteria as NC

auth_token_file = open("/Users/rweberla/flybrain-clustering-rhessa/flybrain.auth.txt", 'r') # need file containing the authentication token
auth_token = next(auth_token_file).strip()
try:
    np_client = Client('neuprint.janelia.org', dataset='hemibrain:' + 'v1.2.1', token=auth_token)
    print('Connected to NeuPrint')
except:
    np_client = None

Connected to NeuPrint


Here I am pulling all neurons from the Hemibrain Database. I might be doing this incorrectly but this seems to produce the most rows of neuron ids

In [3]:
neurons, roi = fetch_neurons(NC())
neurons

/Users/rweberla/Downloads/enter/envs/fbl/lib/python3.12/site-packages/neuprint/client.py:609: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  neuron_props_val = df_results.iloc[0][0]


,bodyId,instance,type,pre,post,downstream,upstream,mito,size,status,cropped,statusLabel,cellBodyFiber,somaRadius,somaLocation,roiInfo,notes,inputRois,outputRois
0,106979579,Franken1,None,3,6,17,6,21,97628357,Unimportant,None,Unimportant,None,NaN,None,"{'PENP': {'pre': 3, 'post': 4, 'downstream': 1...",None,"[AMMC, PENP, SAD]","[AMMC, PENP, SAD]"
1,108632992,Franken2,None,13,55,41,55,932,10146826497,Unimportant,None,Unimportant,None,301.0,"[18418, 11024, 11376]","{'MB(R)': {'pre': 2, 'post': 12, 'downstream':...",None,"[CA(R), GNG, IPS(R), MB(+ACA)(R), MB(R), VMNP]","[CA(R), GNG, IPS(R), MB(+ACA)(R), MB(R), VMNP]"
2,198469830,None,None,0,0,0,0,2,6665418,Traced,True,Leaves,None,299.0,"[33045, 15655, 3280]",{},None,[],[]
3,198810827,None,None,0,0,0,0,1,9211633,Traced,True,Leaves,None,299.0,"[33014, 16221, 3088]",{},None,[],[]
4,198815212,None,None,0,0,0,0,3,27720313,Traced,True,Leaves,None,299.0,"[33342, 16128, 3216]",{},None,[],[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
186056,7112626663,None,None,40,52,351,52,8,73261205,Assign,None,0.5assign,None,NaN,None,"{'SNP(L)': {'pre': 40, 'post': 52, 'downstream...",None,"[SMP(L), SNP(L)]","[SMP(L), SNP(L)]"
186057,7112626669,None,None,7,2,46,2,2,8669132,Assign,None,0.5assign,None,NaN,None,"{'AL(R)': {'pre': 7, 'post': 2, 'downstream': ...",None,"[AL(R), AL-VC5(R)]","[AL(R), AL-VC5(R), AL-VP2(R)]"
186058,7112626688,None,None,0,10,0,10,4,3813327,Assign,None,0.5assign,None,NaN,None,"{'SNP(R)': {'post': 10}, 'SLP(R)': {'post': 10}}",None,"[SLP(R), SNP(R)]",[]
186059,7112626733,None,None,2,26,13,26,2,6115066,Assign,None,0.5assign,None,NaN,None,"{'SNP(R)': {'pre': 2, 'post': 26, 'downstream'...",None,"[SMP(R), SNP(R)]","[SMP(R), SNP(R)]"


Checking that there are lots of neuron types to parse through

In [4]:
len(neurons['type'].unique())

5621

Checking that expected EN type is there

In [5]:
neurons[neurons['type']=='vpoEN']

,bodyId,instance,type,pre,post,downstream,upstream,mito,size,status,cropped,statusLabel,cellBodyFiber,somaRadius,somaLocation,roiInfo,notes,inputRois,outputRois
78495,1570922129,vpoEN_R,vpoEN,517,1063,3751,1063,268,783469177,Traced,False,Roughly traced,AVL13,331.5,"[9256, 25892, 34840]","{'SNP(R)': {'pre': 1, 'post': 1, 'downstream':...",None,"[AMMC, AVLP(R), EPA(R), ICL(R), INP, PENP, PVL...","[AVLP(R), EPA(R), ICL(R), INP, PVLP(R), SCL(R)..."
179651,5813027266,vpoEN_R,vpoEN,518,1013,3693,1013,256,721177071,Traced,False,Roughly traced,AVL13,331.5,"[5731, 27307, 32328]","{'SNP(R)': {'pre': 1, 'downstream': 8}, 'SIP(R...",None,"[AMMC, AVLP(R), EPA(R), ICL(R), INP, PENP, PVL...","[AVLP(R), EPA(R), ICL(R), INP, PVLP(R), SCL(R)..."


Dropping None types and any duplicates

In [6]:
all_types = neurons['type'].dropna()
all_types = all_types.unique().tolist()

This algorithm finds the "root" of each celltype and saves it to a list. This drops all numbers that may be in a generic label.

In [7]:
root_letters = [] # list to store the roots
for t in all_types:
    root = '' # root letters of the type name
    for c in t:
        if c.isalpha(): # gets all letters up to the first number
                root += c
    root_letters.append(root)

# drop duplicates 
root_letters = list(set(root_letters))

In [8]:
len(root_letters)

632

In [9]:
# looking through the root letters and finding the types that contain IN or EN at the end of each root 
# store them in a new list
new_types = []
for r in root_letters:
        if r[-2:] == 'IN' or r[-2:] == 'EN':
            new_types.append(r)

new_types


['vpoEN', 'PENbPEN', 'PENaPEN', 'ALIN', 'oviIN']

In [10]:
# See if vpoIN is in the list of types
'vpoIN' in all_types


False

In [11]:
# See if oviEN is in the list of types
'oviEN' in all_types

False

It seems this process doesn't really work! The vpoIN or oviEN aren't labelled in this data at all.